In [70]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type, add_income_group, add_all_category_columns
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1500
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
add_race_eth(df) # Adds race_eth column
add_purpose_type(df) # Adds purpose_type column Combines loan_purpose and loan_type in the same way as the summary report
add_income_group(df) # Adds income_group column
add_all_category_columns(df) # Add '_c' categorical columns

ts_source = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas

Adding category column for  income_group
Adding category column for  loan_type
Adding category column for  loan_purpose
Adding category column for  race_eth


In [72]:
def get_ts(year):
    ts_name = ts_source[ts_source.activity_year == year][['respondent_id','respondent_name']].copy()
    ts_name.respondent_id = ts_name.respondent_id.astype(str)
    ts_name = ts_name.drop_duplicates(subset=['respondent_id'])
    ts_name = ts_name.drop_duplicates(subset=['respondent_name'])
    return ts_name

ts = get_ts(2012)
df = pd.merge(df, ts, on='respondent_id', how='left')

In [73]:
# Function to create each section of the standard report. 
def standard_report_section(df, column_name, totalcount, totalamount):
    dfo = df.groupby(column_name)
    dfo = dfo['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    dfo['%count'] = np.round(100*dfo['count']/totalcount,1)
    dfo['%amount'] = np.round(100*dfo['amount']/totalamount,1)
    
    return dfo[['median','count','%count','amount','%amount']]

In [74]:
# Calling the standard_report_section
totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()

# List Version
for col_name in ('purpose_type','race_eth','income_group', 'applicant_sex','occupancy'):
    frames.append(standard_report_section(df, col_name, totalcount, totalamount))

# Comprehension version
frames = [standard_report_section(df, column_name, totalcount, totalamount) for column_name in 
         ('purpose_type','race_eth','income_group','applicant_sex','occupancy')]
    
table = pd.concat(frames, keys=['Loan Purpose and Type', 
                               'Applicant Race/Eth', 
                               'Applicant Income',
                               'Applicant Sex',
                               'Occupancy'])

pieces = [table.ix[:,["median","count","%count","amount","%amount",]]]
table = pd.concat(pieces, axis=1)
table

NameError: name 'frames' is not defined